In [1]:
import tensorflow as tf
import pickle
import os
import shutil
import tempfile
import datetime
from glob import iglob
import time
# import attention
from collections import deque
import pickle
import pyreader
import numpy as np
import csv
from batchmake import Batcher
import pprint

In [2]:
datas= 'data_samples/'
word_to_id='data_samples/mapping.map'
with open(word_to_id, 'rb') as f:
    word_to_id = pickle.load(f)
inv_map = {v:k for k, v in word_to_id.items() }
batch_data = datas+'output.txt.part0'
# batch_data = datas+'preprocess.part0'
with open(batch_data, 'rb') as f:
    picked_data = pickle.load(f)

In [3]:
def attention(inputs, attention_size):
    inputs_shape = inputs.shape
    sequence_length = inputs_shape[1].value
    hidden_size = inputs_shape[2].value
    
    w_omega = tf.Variable(tf.random_normal([hidden_size,attention_size], stddev=0.1))
    b_omega = tf.Variable(tf.random_normal([attention_size], stddev=0.1))
    u_omega = tf.Variable(tf.random_normal([attention_size], stddev=0.1))
    
    # batch *k , attention_size
    v = tf.tanh(tf.matmul(tf.reshape(inputs, [-1,hidden_size]), w_omega)+
               tf.reshape(b_omega,[1,-1]))
    
    vu = tf.matmul(v, tf.reshape(u_omega,[-1,1])) # batch*k,1
    exps = tf.reshape(tf.exp(vu), [-1, sequence_length])  # (batch, k)
    alphas = exps / tf.reshape(tf.reduce_sum(exps,1), [-1,1])
    #(batch,k)
    output = tf.reduce_sum(inputs * tf.reshape(alphas, [-1, sequence_length,1]),1)
    # batch * sequence * hidden -> batch * hidden
    return output

In [4]:
hidden_size=32
sequence=5
input_dim=11
attention_size = 50
batch_size=1
grucell = tf.nn.rnn_cell.BasicLSTMCell(hidden_size)
X = tf.placeholder(tf.float32, [None, sequence,input_dim], name='inputs_xdata')
Y = tf.placeholder(tf.float32, [None, sequence, input_dim-1], name='targets_ydata')
with tf.variable_scope('RNN'):
    outputs, states = tf.nn.dynamic_rnn(grucell,X, dtype=tf.float32)

In [5]:
attention_output = attention(outputs, attention_size=attention_size)

In [6]:
keep_prob = 0.5
drop = tf.nn.dropout(attention_output, keep_prob=keep_prob)

w = tf.Variable(tf.truncated_normal([hidden_size,1], stddev=0.1))
b = tf.Variable(tf.constant(0., shape=[1]))
y_hat = tf.nn.xw_plus_b(drop,w,b)
y_hat = tf.squeeze(y_hat)

loss = -tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=y_hat, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=1e-3).minimize(loss)

In [7]:
filename_queue = tf.train.string_input_producer(['./inputs.csv'])
reader = tf.TextLineReader()
key, record_string = reader.read_up_to(filename_queue,num_records=sequence)
# key, record_string = reader.read(filename_queue)
seq_lengths = [[11]]
# record_string = tf.reshape(record_string, [-1])
record_defaults = [[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1]]
batc_record = tf.cast(tf.decode_csv(record_string, record_defaults=record_defaults),tf.int32)
batc_record = tf.reshape(batc_record,[input_dim,sequence])
# target_string = tf.slice(record_string, [1,0],[1,-1])
# file_content = tf.read_file(filename_queue)
# batc_record = tf.transpose(batc_record,[0,2,1])
# output = tf.train.batch([batc_record], batch_size=batch_size, dynamic_pad=True,allow_smaller_final_batch=False)
output = tf.train.shuffle_batch(tensors=[batc_record],
                                batch_size=batch_size,
                                capacity = 5000,
                                num_threads=1,
                                min_after_dequeue=1000
                               )
output = tf.transpose(output,[0,2,1])
target = tf.slice(output, [0,0,1],[-1,-1,-1])
print(output,target)
# output = tf.reshape(output,[2,3,11])
Ses = tf.Session()

# sequences, output = tf.contrib.training.bucket_by_sequence_length(input_length=seq_lengths,
#                                              tensors=[batc_record],
#                                              batch_size=1,
#                                              bucket_boundaries=[11,13],
#                                              dynamic_pad=True
#                                              )

Tensor("transpose:0", shape=(1, 5, 11), dtype=int32) Tensor("Slice:0", shape=(1, 5, 10), dtype=int32)


In [8]:
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=Ses, coord=coord)
Ses.run(tf.global_variables_initializer())

for i in range(1000):
    output_,target_ = Ses.run([output,target])
    feed_data = {X:output_, Y:target_}
#     try:
    lo, opt = Ses.run([loss,optimizer], feed_dict=feed_data)
    print(feed_data)
#     except:
#         print('error occured!!!  ',i, feed_data)
#         break
    print('loss',lo)
# batch_x_ = Ses.run([batch_x])

# print(np.array(batch_x_[0]))
# print(np.array(batch_x_[0]))
# print(batch_x.shape)
# output_,target_=Ses.run([output,target])
# print(np.array(target_))
# print(np.array(target_).shape)

INFO:tensorflow:Error reported to Coordinator: <class 'tensorflow.python.framework.errors_impl.InvalidArgumentError'>, Input to reshape is a tensor with 22 values, but the requested shape has 55
	 [[Node: Reshape_6 = Reshape[T=DT_INT32, Tshape=DT_INT32, _device="/job:localhost/replica:0/task:0/cpu:0"](Cast/x, Reshape_6/shape)]]
{<tf.Tensor 'inputs_xdata:0' shape=(?, 5, 11) dtype=float32>: array([[[  22,  174,   20,  255,    2,   22,   27,    5,  150,    5, 3000],
        [  22,   27,    5,   81,    5,  658,    5,   21,   20,  236,    6],
        [  22,   27,    5,  150,    5, 3000,   20,  150,    2,   22,   27],
        [  22,    1,    5,    1,   20,    1,    2,   22,   27,    5,   83],
        [  20,   37,    2,   22,  174,   20,  255,    2,   22,   27,    5]]]), <tf.Tensor 'targets_ydata:0' shape=(?, 5, 10) dtype=float32>: array([[[ 174,   20,  255,    2,   22,   27,    5,  150,    5, 3000],
        [  27,    5,   81,    5,  658,    5,   21,   20,  236,    6],
        [  27,    5,  1

loss -56.2101
{<tf.Tensor 'inputs_xdata:0' shape=(?, 5, 11) dtype=float32>: array([[[  22,  151,   20,  258,    2,   22,   27,    5,  133,    5,    1],
        [  20,  219,    2,   22, 1503,   20,  832,    2,   22,   27,    5],
        [  22,   27,    5,  133,    5,    1,   20,   65,    2,   22,    1],
        [  22,  535,   20,  259,   64,  796,    2,   22,   27,    5,  133],
        [  20,  255,    2,   20,   59,    2,   20,  355,    2,   22,    1]]]), <tf.Tensor 'targets_ydata:0' shape=(?, 5, 10) dtype=float32>: array([[[ 151,   20,  258,    2,   22,   27,    5,  133,    5,    1],
        [ 219,    2,   22, 1503,   20,  832,    2,   22,   27,    5],
        [  27,    5,  133,    5,    1,   20,   65,    2,   22,    1],
        [ 535,   20,  259,   64,  796,    2,   22,   27,    5,  133],
        [ 255,    2,   20,   59,    2,   20,  355,    2,   22,    1]]])}
loss -105.623
{<tf.Tensor 'inputs_xdata:0' shape=(?, 5, 11) dtype=float32>: array([[[  22,   27,    5,  106,   20, 4538,    2,

        [535,  20, 259,   2,  22,  27,   5,  44,  20,  21]]])}
loss -15.8716
{<tf.Tensor 'inputs_xdata:0' shape=(?, 5, 11) dtype=float32>: array([[[    1,     2,    20,    59,     2,  4141,     7,     3,     1,
             6,     1],
        [    1,     2,    20, 13134,     2,    20,    59,     2,    22,
           862,    20],
        [   20,    59,     2,    22,    37,    20,    37,     2,    22,
          1841,     5],
        [    1,     2,    20,    59,     2,    20,   701,     2,    22,
            27,     5],
        [    1,     2,    20,    59,     2,    22,   525,    20,   815,
             2,    22]]]), <tf.Tensor 'targets_ydata:0' shape=(?, 5, 10) dtype=float32>: array([[[    2,    20,    59,     2,  4141,     7,     3,     1,     6,
             1],
        [    2,    20, 13134,     2,    20,    59,     2,    22,   862,
            20],
        [   59,     2,    22,    37,    20,    37,     2,    22,  1841,
             5],
        [    2,    20,    59,     2,    20,   701

loss -147.309
{<tf.Tensor 'inputs_xdata:0' shape=(?, 5, 11) dtype=float32>: array([[[  20,    1,    2,   22,  525,   20,  815,    2,   22, 9762,    5],
        [  22,   27,    5,  182,    5,  226,   20,   52,    2,   22,   27],
        [  22,   27,    5,   81,    5,  206,    5,   44,   20,   21,    2],
        [   1,    2,   20,   59,    2,   22,  174,   20, 1276,    6,  678],
        [  20,  174,    2,   22,   27,    5,   81,    5,  206,    5,  654]]]), <tf.Tensor 'targets_ydata:0' shape=(?, 5, 10) dtype=float32>: array([[[   1,    2,   22,  525,   20,  815,    2,   22, 9762,    5],
        [  27,    5,  182,    5,  226,   20,   52,    2,   22,   27],
        [  27,    5,   81,    5,  206,    5,   44,   20,   21,    2],
        [   2,   20,   59,    2,   22,  174,   20, 1276,    6,  678],
        [ 174,    2,   22,   27,    5,   81,    5,  206,    5,  654]]])}
loss -386.091
{<tf.Tensor 'inputs_xdata:0' shape=(?, 5, 11) dtype=float32>: array([[[  22,   27,    5,   44,   20,  622,    2,

{<tf.Tensor 'inputs_xdata:0' shape=(?, 5, 11) dtype=float32>: array([[[ 22,  27,   5,  83,  20,  75,   2,  22, 535,  20, 259],
        [ 22,  27,   5,  83,  20,  75,   2,  22,   1,   5, 150],
        [ 22,  27,   5,  83,  20,  75,   2,  22, 535,  20, 259],
        [ 22, 535,  20, 259,  64, 796,   2,  22,  27,   5,  83],
        [ 22,  27,   5,  83,  20,  75,   2,  22,   1,   5, 150]]]), <tf.Tensor 'targets_ydata:0' shape=(?, 5, 10) dtype=float32>: array([[[ 27,   5,  83,  20,  75,   2,  22, 535,  20, 259],
        [ 27,   5,  83,  20,  75,   2,  22,   1,   5, 150],
        [ 27,   5,  83,  20,  75,   2,  22, 535,  20, 259],
        [535,  20, 259,  64, 796,   2,  22,  27,   5,  83],
        [ 27,   5,  83,  20,  75,   2,  22,   1,   5, 150]]])}
loss -81.3092
{<tf.Tensor 'inputs_xdata:0' shape=(?, 5, 11) dtype=float32>: array([[[  22,   27,    5,   44,   20,   21,    2,    2,    2,   23, 3973],
        [  22,   27,    5,   44,   20,   61,    6,   21,    2,    2,    2],
        [  22,   

...

{<tf.Tensor 'inputs_xdata:0' shape=(?, 5, 11) dtype=float32>: array([[[   22,    27,     5,   181,     5,   787,    20, 10266,     2,
            22,    27],
        [   22,    27,     5,   181,     5,   787,    20, 11249,     2,
            22,    27],
        [   22,    27,     5,   181,     5,   787,    20,     1,     2,
            22,    27],
        [   22,    27,     5,   181,     5,   787,    20,   484,     2,
            22,    27],
        [   22,    27,     5,   181,     5,   787,    20, 12349,     2,
            22,    27]]]), <tf.Tensor 'targets_ydata:0' shape=(?, 5, 10) dtype=float32>: array([[[   27,     5,   181,     5,   787,    20, 10266,     2,    22,
            27],
        [   27,     5,   181,     5,   787,    20, 11249,     2,    22,
            27],
        [   27,     5,   181,     5,   787,    20,     1,     2,    22,
            27],
        [   27,     5,   181,     5,   787,    20,   484,     2,    22,
            27],
        [   27,     5,   181,     5, 

{<tf.Tensor 'inputs_xdata:0' shape=(?, 5, 11) dtype=float32>: array([[[   1,    2,   22,  535,   20,  259,    2,   22,   27,    5,   81],
        [  22,   27,    5,   81,    5,  206,    5,   44,    5,   21,    5],
        [  20,  694,    2,   22,   27,    5,  182,   20,   78,    2,   22],
        [  22,   27,    5,  106,   20,   72,    2,   22,    5,  137,   20],
        [  22,   27,    5,   81,    5,  206,    5,  428,    5, 1035,   20]]]), <tf.Tensor 'targets_ydata:0' shape=(?, 5, 10) dtype=float32>: array([[[   2,   22,  535,   20,  259,    2,   22,   27,    5,   81],
        [  27,    5,   81,    5,  206,    5,   44,    5,   21,    5],
        [ 694,    2,   22,   27,    5,  182,   20,   78,    2,   22],
        [  27,    5,  106,   20,   72,    2,   22,    5,  137,   20],
        [  27,    5,   81,    5,  206,    5,  428,    5, 1035,   20]]])}
loss -428.213
{<tf.Tensor 'inputs_xdata:0' shape=(?, 5, 11) dtype=float32>: array([[[   20,   191,     2,    20, 10856,     2,    22,    27,

             2]]])}
loss -2885.77
{<tf.Tensor 'inputs_xdata:0' shape=(?, 5, 11) dtype=float32>: array([[[   1,    2,   20,  535,    2,   20, 3451,    2,   20,  191,    2],
        [  20, 1656,    2,   22,   27,    5,   44,    5,  403,    5,  160],
        [  20,   59,    2,   20,  862,    2,   20,  219,    2,   22,   27],
        [  22,   27,    5,   44,   20,   73,    2,   22,   27,    5,   44],
        [  20,  191,    2,   20,  355,    2,   22,   27,    5,   44,    5]]]), <tf.Tensor 'targets_ydata:0' shape=(?, 5, 10) dtype=float32>: array([[[   2,   20,  535,    2,   20, 3451,    2,   20,  191,    2],
        [1656,    2,   22,   27,    5,   44,    5,  403,    5,  160],
        [  59,    2,   20,  862,    2,   20,  219,    2,   22,   27],
        [  27,    5,   44,   20,   73,    2,   22,   27,    5,   44],
        [ 191,    2,   20,  355,    2,   22,   27,    5,   44,    5]]])}
loss -1011.68
{<tf.Tensor 'inputs_xdata:0' shape=(?, 5, 11) dtype=float32>: array([[[ 20, 191,   2,  22, 6

{<tf.Tensor 'inputs_xdata:0' shape=(?, 5, 11) dtype=float32>: array([[[22, 27,  5, 44, 20, 61,  6, 21,  2,  2,  2],
        [22, 27,  5, 44, 20, 61,  6, 21,  2,  2,  2],
        [22, 27,  5, 44, 20, 61,  6, 21,  2,  2,  2],
        [22, 27,  5, 44, 20, 61,  6, 21,  2,  2,  2],
        [22, 27,  5, 44, 20, 61,  6, 21,  2,  2,  2]]]), <tf.Tensor 'targets_ydata:0' shape=(?, 5, 10) dtype=float32>: array([[[27,  5, 44, 20, 61,  6, 21,  2,  2,  2],
        [27,  5, 44, 20, 61,  6, 21,  2,  2,  2],
        [27,  5, 44, 20, 61,  6, 21,  2,  2,  2],
        [27,  5, 44, 20, 61,  6, 21,  2,  2,  2],
        [27,  5, 44, 20, 61,  6, 21,  2,  2,  2]]])}
loss -125.868
{<tf.Tensor 'inputs_xdata:0' shape=(?, 5, 11) dtype=float32>: array([[[   1,    2,   22,   27,    5,  133,    5,  587,    5,  403,    5],
        [  20,    1,    2,   20, 1725,    2,   20,    1,    2,   20,   59],
        [  22,   27,    5,  133,   20,  587,    2,   22,   27,    5,  133],
        [   1,    2,   22,   27,    5,   81,  

OutOfRangeError: RandomShuffleQueue '_1_shuffle_batch/random_shuffle_queue' is closed and has insufficient elements (requested 1, current size 0)
	 [[Node: shuffle_batch = QueueDequeueManyV2[component_types=[DT_INT32], timeout_ms=-1, _device="/job:localhost/replica:0/task:0/cpu:0"](shuffle_batch/random_shuffle_queue, shuffle_batch/n)]]

Caused by op 'shuffle_batch', defined at:
  File "D:\anaconda_python\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "D:\anaconda_python\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "D:\anaconda_python\lib\site-packages\ipykernel\__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "D:\anaconda_python\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "D:\anaconda_python\lib\site-packages\ipykernel\kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "D:\anaconda_python\lib\site-packages\zmq\eventloo

In [6]:
keep_prob = 0.5
drop = tf.nn.dropout(attention_output, keep_prob=keep_prob)

w = tf.Variable(tf.truncated_normal([hidden_size,1], stddev=0.1))
b = tf.Variable(tf.constant(0., shape=[1]))
y_hat = tf.nn.xw_plus_b(drop,w,b)
y_hat = tf.squeeze(y_hat)

loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=y_hat, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=1e-3).minimize(loss)

Tensor("Sum_1:0", shape=(?, 32), dtype=float32)


In [7]:
s = tf.Session()

In [8]:
batcher = Batcher([batch_data], batch_size=batch_size, seq_length=sequence) #batchsize, sequence

for batch in batcher:
    for i, feed_data  in enumerate(batcher.sequence_iterator(batch)):
        inputs,targets,masks,identifiers,actual_lengths = feed_data
        print(inputs.shape , targets.shape, masks.shape, identifiers.shape,actual_lengths.shape)
        print(actual_lengths)
        
        feed_data = {X:inputs, Y:targets}
        #inputs :  batch, sequence
        
        lo, opt = s.run([loss,optimizer], feed_dict=feed_data)
        
            
    break

Loaded data partition data_samples/preprocess.part0 with 481 examples
[<pyreader.Container object at 0x000001C9A750A518>, <pyreader.Container object at 0x000001C9A78A1898>, <pyreader.Container object at 0x000001C9A79122B0>, <pyreader.Container object at 0x000001C9A79D2BA8>, <pyreader.Container object at 0x000001C9A7953358>, <pyreader.Container object at 0x000001C9A79803C8>, <pyreader.Container object at 0x000001C9A79EADD8>, <pyreader.Container object at 0x000001C9A7A06978>, <pyreader.Container object at 0x000001C9A7A12278>, <pyreader.Container object at 0x000001C9A7A12390>, <pyreader.Container object at 0x000001C9A7A124A8>, <pyreader.Container object at 0x000001C9A7A125C0>, <pyreader.Container object at 0x000001C9A7A126D8>, <pyreader.Container object at 0x000001C9A7A127F0>, <pyreader.Container object at 0x000001C9A7A12908>, <pyreader.Container object at 0x000001C9A7A12A20>]
(5, 16) (5, 16) (5, 16, 1) (16, 5) (16,)
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]


ValueError: Cannot feed value of shape (5, 16) for Tensor 'Placeholder:0', which has shape '(?, 5, 1)'